In [ ]:
import tensorflow as tf
import os
import numpy as np

from tensorflow.keras.applications import VGG19
from tensorflow.keras import layers
from tensorflow.keras import Model


In [ ]:
print(os.getcwd())  # to get the current working directory

In [ ]:
# code only necessary when using colab to extract the dataset from the uploaded dataset.zip file

file_path = '/dataset.zip'
from zipfile import ZipFile
file_name = file_path

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

In [ ]:
# Using VGG19 model with Imagenet weights to extract features from the images for faster and better results.

base_model = VGG19(include_top=False, weights='imagenet', input_shape=(460,240,3))

for layer in base_model.layers:    # freezing the base_model layers
    layer.trainable= False
    
# base_model.summary()

last_output = base_model.output

x = layers.Flatten()(last_output)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(64, activation='relu')(x)                
x = layers.Dense(2, activation='softmax')(x)    # number of neurons to be changed depending upon number of output classes       

model = Model( base_model.input, x) 

model.compile(optimizer ='adam', 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

model.summary()

In [ ]:
dir_path = "/content/dataset"  # dataset path

batch_size = 128

image_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=0.4, shear_range=0.2, fill_mode='nearest', horizontal_flip=True,
    vertical_flip=False, rescale=1./255, preprocessing_function=None,
    validation_split=0.2
)

train_data = image_generator.flow_from_directory(
    dir_path,
    target_size = (460,240),
    batch_size = batch_size,
    class_mode = 'categorical',
    subset = "training"
)

validation_data = image_generator.flow_from_directory(
    dir_path,
    target_size = (460,240),
    batch_size = batch_size,
    class_mode = 'categorical',
    subset = "validation"
)


In [ ]:
# only used for basic testing and comparision.

# model = tf.keras.Sequential([
#     tf.keras.layers.Conv2D(32,(5,5), activation='relu', input_shape=(460,240,3)),
#     tf.keras.layers.MaxPooling2D(4),
#     tf.keras.layers.Conv2D(64,(5,5), activation='relu'),
#     tf.keras.layers.MaxPooling2D(4),
#     tf.keras.layers.Conv2D(64,(5,5), activation='relu'),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(512,activation='relu'),
#     tf.keras.layers.Dense(128,activation='relu'),
#     tf.keras.layers.Dense(64, activation='relu'),
#     tf.keras.layers.Dense(2,activation='softmax'),
# ])

# model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['acc'])

In [ ]:
# to be used to reset the GPU if required

# from numba import cuda 
# device = cuda.get_current_device()
# device.reset()


In [ ]:
# for training using CPU

# with tf.device('/CPU:0'):
#     history = model.fit(train_data, epochs=10, validation_data = validation_data, verbose=2 )


In [ ]:
# for training using GPU

history = model.fit(train_data, epochs=10, validation_data = validation_data )

In [ ]:
# to be used to save the model after training

# path = 'C:\Users\sinha\Desktop\Open cv\saved_model'
# model.save(path)
    